## Scrape the Wikipedia page

In [154]:
from lxml import etree
import urllib.request

# Setup webpage downloader
with urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as url:
    s = url.read()
    #print(s)

html = etree.HTML(s)

## Get all 'tr'
tr_nodes = html.xpath('//table[@class="wikitable sortable"]/tbody/tr')

## Get text from rest all 'td'
td_content = [[td.text for td in tr.xpath('td')] for tr in tr_nodes[1:]]
#print(td_content)

# Organize data
postal_code = []
borough = []
neighborhood = []
all_lists = []

for x in range(len(td_content)):
    if td_content[x][1][:-1] != "Not assigned" and td_content[x][2][:-1] != "Not assigned":
        postal_code.append(td_content[x][0][:-2])
        borough.append(td_content[x][1][:-1])
        neighborhood.append(td_content[x][2][:-1])
    
print(postal_code)
print(borough)
        

['M3', 'M4', 'M5', 'M6', 'M7', 'M9', 'M1', 'M3', 'M4', 'M5', 'M6', 'M9', 'M1', 'M3', 'M4', 'M5', 'M6', 'M9', 'M1', 'M4', 'M5', 'M6', 'M1', 'M4', 'M5', 'M6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M9', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M9', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M9', 'M1', 'M2', 'M4', 'M5', 'M6', 'M9', 'M1', 'M2', 'M4', 'M5', 'M6', 'M7', 'M9', 'M1', 'M4', 'M5', 'M6', 'M1', 'M4', 'M5', 'M1', 'M4', 'M5', 'M8', 'M9', 'M1', 'M4', 'M5', 'M8', 'M9', 'M1', 'M4', 'M5', 'M8', 'M4', 'M7', 'M8', 'M8']
['North York', 'North York', 'Downtown Toronto', 'North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough', 'North York', 'East York', 'Downtown Toronto', 'North York', 'Etobicoke', 'Scarborough', 'North York', 'East York', 'Downtown Toronto', 'York', 'Etobicoke', 'Scarborough', 'East Toronto', 'Downtown Toronto', 'York', 'Scarborough', 'East York', 'Downtown Toronto', 'Downt

In [155]:
# Setup pandas dataframe
import pandas as pd
drop_list = []

Neighborhood = pd.DataFrame({"PostalCode":postal_code,"Borough":borough,"Neighborhood":neighborhood})
Neighborhood


,PostalCode,Borough,Neighborhood
0,M3,North York,Parkwoods
1,M4,North York,Victoria Village
2,M5,Downtown Toronto,"Regent Park, Harbourfront"
3,M6,North York,"Lawrence Manor, Lawrence Heights"
4,M7,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4,Downtown Toronto,Church and Wellesley
100,M7,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [156]:
# Unnecessarily clarifying shape that can be seen just running the above cell for exam purposes
Neighborhood.shape

(103, 3)

## Get coordinates using Geocoder package

In [167]:
import geocoder # Unfortunately this package doesn't work. It just returns NoneTypes so i used the CSV instead
csv = pd.read_csv("https://cocl.us/Geospatial_data")

print(csv)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [173]:
# Add rows for new data
Neighborhood['Latitude'] = ""
Neighborhood['Longitude'] = ""

# Get coordinates from PCs
for index, row in csv.iterrows():
    for sub_index, sub_row in Neighborhood.iterrows():
        if row['Postal Code'][:-1] == sub_row['PostalCode']:
            sub_row['Latitude'] =  row['Latitude']
            sub_row['Longitude'] =  row['Longitude']
            
Neighborhood

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3,North York,Parkwoods,43.7616,-79.521
1,M4,North York,Victoria Village,43.6659,-79.3832
2,M5,Downtown Toronto,"Regent Park, Harbourfront",43.6484,-79.3823
3,M6,North York,"Lawrence Manor, Lawrence Heights",43.6516,-79.4844
4,M7,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6627,-79.3216
...,...,...,...,...,...
98,M8,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6288,-79.521
99,M4,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7,East Toronto,"Business reply mail Processing Centre, South C...",43.6627,-79.3216
101,M8,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6288,-79.521


## Explore and cluster the neighborhoods in Toronto

In [179]:
# Downloading/importing packages
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Some informations about our dataframe and our place
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Neighborhood['Borough'].unique()),
        Neighborhood.shape[0]
    )
)

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornto are {}, {}.'.format(latitude, longitude))

The dataframe has 10 boroughs and 103 neighborhoods.
The geograpical coordinate of Tornto are 43.6534817, -79.3839347.


### Create a map of Tornto

In [184]:
import folium # map rendering library

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Neighborhood['Latitude'], Neighborhood['Longitude'], Neighborhood['Borough'], Neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Log into Foursquare and explore Toronto's venues

In [185]:
CLIENT_ID = 'QEKVYOHVSMWOW12V2S1T3HIRUQP3B02W342Z2TWNTMQH20EP' # your Foursquare ID
CLIENT_SECRET = '344DWOTSL3IKCM4B1XRFSZ2MTHWR1WRBTIB3BKCW0RE3PLCK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QEKVYOHVSMWOW12V2S1T3HIRUQP3B02W342Z2TWNTMQH20EP
CLIENT_SECRET:344DWOTSL3IKCM4B1XRFSZ2MTHWR1WRBTIB3BKCW0RE3PLCK


In [192]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [194]:
toronto_venues = getNearbyVenues(names=Neighborhood['Neighborhood'],
                                   latitudes=Neighborhood['Latitude'],
                                   longitudes=Neighborhood['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [195]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 118 uniques categories.


### Analyze each neighborhood

In [197]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

# Group neighborhoods
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Bakery,Bank,...,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Alderwood, Long Branch",0.000000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.066667,0.000000,...,0.066667,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.066667,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.00,0.000000,0.00,0.25,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
2,Bayview Village,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.000000,0.166667,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.000000,0.030000,0.01,0.000000,0.03,0.00,0.0,0.010000,0.000000,...,0.000000,0.02,0.020000,0.010000,0.000000,0.01,0.01,0.01,0.000000,0.000000
4,Berczy Park,0.000000,0.030000,0.01,0.000000,0.03,0.00,0.0,0.010000,0.000000,...,0.000000,0.02,0.020000,0.010000,0.000000,0.01,0.01,0.01,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,"Willowdale, Willowdale East",0.000000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.000000,0.166667,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
78,"Willowdale, Willowdale West",0.000000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.000000,0.166667,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000
79,Woodbine Heights,0.013333,0.013333,0.00,0.013333,0.00,0.00,0.0,0.000000,0.000000,...,0.000000,0.00,0.013333,0.013333,0.013333,0.00,0.00,0.00,0.000000,0.026667
80,York Mills West,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.000000,0.166667,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000


In [198]:
# Print most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
                  venue  freq
0        Hardware Store  0.07
1  Fast Food Restaurant  0.07
2           Social Club  0.07
3        Sandwich Place  0.07
4          Burger Joint  0.07


----Bathurst Manor, Wilson Heights, Downsview North----
                  venue  freq
0  Gym / Fitness Center  0.25
1        Discount Store  0.25
2    Athletics & Sports  0.25
3         Grocery Store  0.25
4     Afghan Restaurant  0.00


----Bayview Village----
           venue  freq
0           Bank  0.17
1        Butcher  0.17
2  Grocery Store  0.17
3    Pizza Place  0.17
4       Pharmacy  0.17


----Bedford Park, Lawrence Manor East----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.08
2                Hotel  0.05
3                  Gym  0.04
4  Japanese Restaurant  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.08
2                Hotel  0.05
3                  Gym  0.04
4 

In [199]:
# Sort in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [204]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Discount Store,Tanning Salon,Fast Food Restaurant,Burger Joint,Kids Store,Flower Shop,Social Club,Supplement Shop,Sandwich Place,Bakery
1,"Bathurst Manor, Wilson Heights, Downsview North",Gym / Fitness Center,Grocery Store,Athletics & Sports,Discount Store,Flower Shop,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run
2,Bayview Village,Grocery Store,Bank,Pharmacy,Coffee Shop,Butcher,Pizza Place,Gay Bar,Falafel Restaurant,Greek Restaurant,Dessert Shop
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Steakhouse,Seafood Restaurant,Deli / Bodega,Asian Restaurant
4,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Steakhouse,Seafood Restaurant,Deli / Bodega,Asian Restaurant


## Cluster neighborhoods

In [208]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 0, 1, 1, 1, 1, 1, 1, 1])

In [214]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3,North York,Parkwoods,43.7616,-79.521,4.0,Gym / Fitness Center,Grocery Store,Athletics & Sports,Discount Store,Flower Shop,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run
1,M4,North York,Victoria Village,43.6659,-79.3832,1.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Mediterranean Restaurant,Pizza Place,Pub,Hotel
2,M5,Downtown Toronto,"Regent Park, Harbourfront",43.6484,-79.3823,1.0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Steakhouse,Seafood Restaurant,Deli / Bodega,Asian Restaurant
3,M6,North York,"Lawrence Manor, Lawrence Heights",43.6516,-79.4844,1.0,Café,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Sushi Restaurant,Fish & Chips Shop,Falafel Restaurant,Indie Movie Theater,Health Food Store
4,M7,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6627,-79.3216,1.0,Light Rail Station,Smoke Shop,Fast Food Restaurant,Pizza Place,Comic Shop,Restaurant,Burrito Place,Skate Park,Farmers Market,Brewery


In [219]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float